In [1]:
import logging
from functools import wraps

from telegram import utils
from telegram import (ReplyKeyboardMarkup, KeyboardButton, InlineKeyboardButton,
                      ReplyKeyboardRemove, InlineKeyboardMarkup,
                      ChatAction, ParseMode)
from telegram.ext import (Updater, CommandHandler, CallbackQueryHandler,
                          MessageHandler, Filters, 
                          RegexHandler, ConversationHandler)

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)

logger = logging.getLogger(__name__)

In [2]:
def send_typing_action(func):
    
    @wraps(func)
    def command_func(bot, update, *args, **kwargs):
        bot.send_chat_action(chat_id=update.effective_message.chat_id, action=ChatAction.TYPING)
        return func(bot, update, *args, **kwargs)
        
    return command_func

In [3]:
cota_chats = {}

In [4]:
class Cota:
    def __init__(self, _id, name, value=None):
        self._id = _id
        self.name = name
        self.value = value
        self.going = []
        
class CotaView:
    def __init__(self, cota):
        self.cota = cota
        
    def btn(self):
        val = '' if not self.cota.value else ' - R$ {:.2f}'.format(self.cota.value)
        btn_text = '{}'.format(self.cota.name) + val
        return InlineKeyboardButton(btn_text, callback_data='show_cota {}'.format(self.cota._id))

In [5]:
# All possible Interactive Boxes States

class MainListState:
    def __init__(self, iBox):
        self.iBox = iBox
        
    def update(self, bot):
        header = 'Lista de Cotas:'
        if not self.iBox.cota_chat.all_cotas:
            header = '*Não tem nenhuma cota!*'
        cota_views = [CotaView(cota) for cota in self.iBox.cota_chat.all_cotas.values()]
        button_list = [cota_view.btn() for cota_view in cota_views]
        new_cota_button = InlineKeyboardButton('Nova Cota', callback_data='new_cota')
        
        menu = [[b] for b in button_list] + [new_cota_button]
        
        bot.edit_message_text(header, 
                              reply_markup=InlineKeyboardMarkup(menu),
                              chat_id=self.iBox.cota_chat._id, 
                              message_id=self.iBox.message_id, 
                              parse_mode=ParseMode.MARKDOWN)

In [6]:
class InteractiveBox:
    def __init__(self, cota_chat):
        self.message_id = None
        self.cota_chat = cota_chat
        
        self.current_state = MainListState(self)
    
    def update(self, bot):
        if not self.message_id:
            message = bot.send_message(self.cota_chat._id, "_Pera aí..._", parse_mode=ParseMode.MARKDOWN)
            self.message_id = message.message_id
        self.current_state.update(bot)

class CotaChat:
    def __init__(self, _id):
        self._id = _id
        self.interactive_chat_boxes = []
        
        self.next_cota_id = 0
        self.all_cotas = {}
        
        self.temp_cota = None
        
    def new_interactive_box(self, bot):
        iBox = InteractiveBox(self)
        self.interactive_chat_boxes.append(iBox)
        iBox.update(bot)
        
    def update(self, bot):
        for icb in self.interactive_chat_boxes:
            icb.update(bot)
            
    def start_temp_cota(self, name):
        self.temp_cota = Cota(self.next_cota_id, name)
            
    def submit_temp_cota(self):
        self.all_cotas[self.temp_cota._id] = self.temp_cota
        self.temp_cota = None
        self.next_cota_id += 1
    

In [13]:
def get_cota_chat(update):
    chat_id = update.message.chat.id
    
    # Add this chat if not present
    if chat_id not in cota_chats:
        cota_chats[chat_id] = CotaChat(chat_id)
        
    return cota_chats[chat_id]

@send_typing_action
def cotas(bot, update):
    cota_chat = get_cota_chat(update)
    cota_chat.new_interactive_box(bot)
    
@send_typing_action
def new_cota(bot, update):
    cota_chat = get_cota_chat(update)
    bot.send_message(cota_chat._id, 'Ok! Qual o nome da cotinha? (/cancel para cancelar)')
    
    return 0

@send_typing_action
def cota_name(bot, update):
    cota_chat = get_cota_chat(update)
    cota_name = update.message.text
    
    cota_chat.start_temp_cota(cota_name)
    bot.send_message(cota_chat._id, 'Blz, qual o valor dela? (/skip pra colocar o valor depois, ou /cancel para cancelar)')
    
    return 1

def cota_value(bot, update):
    cota_chat = get_cota_chat(update)
    cota_value = float(update.message.text)
    
    cota_chat.temp_cota.value = cota_value
    return submit_new_cota(bot, update)
    
@send_typing_action  
def submit_new_cota(bot, update):
    cota_chat = get_cota_chat(update)
    cota_chat.submit_temp_cota()
    
    bot.send_message(cota_chat._id, 'Cota criada!')
    cota_chat.update(bot)
    
    return ConversationHandler.END
    
    
def cancel_new_cota(bot, update):
    cota_chat = get_cota_chat(update)
    cota_chat.temp_cota = None
    
    bot.send_message(cota_chat._id, 'Criação de nova cota cancelada')
    
    return ConversationHandler.END

def callback_handler(bot, update):
    query = update.callback_query
    
    request, data = query.data.split()
    
    if request == 'show_cota':
        print('showing cota {}'.format(data))
    elif request == 'new_cota':
        return new_cota_start(bot, update)
        
    return ConversationHandler.END

def error(bot, update, error):
    """Log Errors caused by Updates."""
    logger.warning('%s', error)

In [14]:
def main():
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater("692336058:AAGFMBpvydprPwlYgQjwMM1QK66oH41qXfA")

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # add handlers
    dp.add_handler(CommandHandler('cotas', cotas))
    
    new_cota_handler = ConversationHandler(
        entry_points=[CommandHandler('newcota', new_cota)],
        states={0: [MessageHandler(Filters.text, cota_name)],
               1: [RegexHandler('\d+(.\d)?', cota_value),
                  CommandHandler('skip', submit_new_cota)]},
        fallbacks=[CommandHandler('cancel', cancel_new_cota)]
    )
    dp.add_handler(new_cota_handler)
    
    dp.add_handler(CallbackQueryHandler(callback_handler))

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

In [ ]:
if __name__ == '__main__':
    main()

2019-03-07 10:34:15,246 - telegram.ext.dispatcher - ERROR - An uncaught error was raised while processing the update
Traceback (most recent call last):
  File "/home/ericklt/miniconda3/lib/python3.6/site-packages/telegram/ext/dispatcher.py", line 279, in process_update
    handler.handle_update(update, self)
  File "/home/ericklt/miniconda3/lib/python3.6/site-packages/telegram/ext/commandhandler.py", line 173, in handle_update
    return self.callback(dispatcher.bot, update, **optional_args)
  File "<ipython-input-2-b73d9d065698>", line 6, in command_func
    return func(bot, update, *args, **kwargs)
  File "<ipython-input-13-a52f3e2cd774>", line 13, in cotas
    cota_chat.new_interactive_box(bot)
  File "<ipython-input-6-a0ada01bc406>", line 27, in new_interactive_box
    iBox.update(bot)
  File "<ipython-input-6-a0ada01bc406>", line 12, in update
    self.current_state.update(bot)
  File "<ipython-input-5-28b6da479a00>", line 21, in update
    parse_mode=ParseMode.MARKDOWN)
  File "/